In [200]:
#Importamos las librerias necesarias para crear la red neuronal
import pandas
import numpy as np
from sklearn import model_selection, preprocessing
from tensorflow import keras
#Leemos el csv
evaluation = pandas.read_csv('turkiye-student-evaluation_R_Specific.csv', header=0)

In [201]:
#Mostramos las 10 primeras lineas para ver que se ha leido bién
evaluation.head(10)

,instr,class,nb.repeat,attendance,difficulty,Q1,Q2,Q3,Q4,Q5,...,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q27,Q28
1,1,2,1,0,4,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
2,1,2,1,1,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
3,1,2,1,2,4,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
4,1,2,1,1,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
5,1,2,1,0,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
6,1,2,1,3,3,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
7,1,2,1,1,3,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
8,1,2,1,1,3,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
9,1,2,1,1,3,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
10,1,2,1,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4


In [202]:
#Seleccionamos los atributos 
atributos = evaluation.loc[:, 'class':'Q27']
atributos = atributos.to_numpy()
scaler = preprocessing.MinMaxScaler()
atributos = scaler.fit_transform(atributos)
print(atributos)

[[0.08333333 0.         0.         ... 0.5        0.5        0.5       ]
 [0.08333333 0.         0.25       ... 0.5        0.5        0.5       ]
 [0.08333333 0.         0.5        ... 1.         1.         1.        ]
 ...
 [1.         0.         0.         ... 1.         1.         1.        ]
 [1.         0.         0.25       ... 0.         0.         0.        ]
 [1.         0.         0.25       ... 0.         0.         0.        ]]


In [203]:
objetivo = evaluation['Q28']
objetivo = evaluation['Q28'].to_numpy(dtype=np.float32)
scaler = preprocessing.MinMaxScaler()
objetivo = scaler.fit_transform(objetivo.reshape(-1, 1))
print(objetivo)

[[0.5]
 [0.5]
 [1. ]
 ...
 [1. ]
 [0. ]
 [0. ]]


In [204]:
(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = model_selection.train_test_split(
    atributos, objetivo, test_size=.15)

In [205]:
normalizador = keras.layers.Normalization()
normalizador.adapt(atributos_entrenamiento)
red_evaluation = keras.Sequential()
red_evaluation.add(keras.layers.Input(shape=(31,)))
red_evaluation.add(normalizador)
red_evaluation.add(keras.layers.Dense(75, activation='relu'))
red_evaluation.add(keras.layers.Dense(75, activation='sigmoid'))
red_evaluation.add(keras.layers.Dense(1, activation='sigmoid'))
red_evaluation.add(keras.layers.Lambda(lambda x: x * 5))


In [206]:
red_evaluation.summary()

Model: "sequential_34"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_34 (Normaliza  (None, 31)               63        
 tion)                                                           
                                                                 
 dense_134 (Dense)           (None, 75)                2400      
                                                                 
 dense_135 (Dense)           (None, 75)                5700      
                                                                 
 dense_136 (Dense)           (None, 1)                 76        
                                                                 
 lambda_13 (Lambda)          (None, 1)                 0         
                                                                 
Total params: 8,239
Trainable params: 8,176
Non-trainable params: 63
__________________________________________________

In [207]:

red_evaluation.compile(optimizer='SGD', loss='mean_squared_error', metrics=['accuracy', 'mae'])
red_evaluation.fit(atributos_entrenamiento, objetivo_entrenamiento,
                batch_size=256, epochs=10)

Epoch 1/10
20/20 [==============================] - 0s 1ms/step - loss: 0.2987 - accuracy: 0.2068 - mae: 0.3653
Epoch 2/10
20/20 [==============================] - 0s 997us/step - loss: 0.0638 - accuracy: 0.3151 - mae: 0.2067
Epoch 3/10
20/20 [==============================] - 0s 1ms/step - loss: 0.0527 - accuracy: 0.3186 - mae: 0.1842
Epoch 4/10
20/20 [==============================] - 0s 1ms/step - loss: 0.0452 - accuracy: 0.3208 - mae: 0.1663
Epoch 5/10
20/20 [==============================] - 0s 1ms/step - loss: 0.0399 - accuracy: 0.3208 - mae: 0.1517
Epoch 6/10
20/20 [==============================] - 0s 977us/step - loss: 0.0364 - accuracy: 0.3224 - mae: 0.1405
Epoch 7/10
20/20 [==============================] - 0s 1ms/step - loss: 0.0339 - accuracy: 0.3228 - mae: 0.1320
Epoch 8/10
20/20 [==============================] - 0s 1ms/step - loss: 0.0320 - accuracy: 0.3226 - mae: 0.1256
Epoch 9/10
20/20 [==============================] - 0s 1ms/step - loss: 0.0307 - accuracy: 0.3228 - 

In [208]:
red_evaluation.evaluate(atributos_prueba, objetivo_prueba)

28/28 [==============================] - 0s 713us/step - loss: 0.0305 - accuracy: 0.3333 - mae: 0.1201


[0.030451636761426926, 0.3333333432674408, 0.12010810524225235]

In [209]:
import pandas
import numpy as np
from sklearn import model_selection, preprocessing
from tensorflow import keras
steel = pandas.read_csv('turkiye-student-evaluation_generic.csv', header=0)
steel.head(10)
atributos = evaluation.loc[:, 'class':'Q27']
atributos = atributos.to_numpy()
scaler = preprocessing.MinMaxScaler()
atributos = scaler.fit_transform(atributos)
print(atributos)
objetivo = evaluation['Q28']
objetivo = evaluation['Q28'].to_numpy(dtype=np.float32)
scaler = preprocessing.MinMaxScaler()
objetivo = scaler.fit_transform(objetivo.reshape(-1, 1))
print(objetivo)
print(objetivo)
(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = model_selection.train_test_split(
    atributos, objetivo, test_size=.10)
normalizador = keras.layers.Normalization()
normalizador.adapt(atributos_entrenamiento)
red_evaluation = keras.Sequential()
red_evaluation.add(keras.layers.Input(shape=(31,)))
red_evaluation.add(normalizador)
red_evaluation.add(keras.layers.Dense(75, activation='relu'))
red_evaluation.add(keras.layers.Dense(75, activation='sigmoid'))
red_evaluation.add(keras.layers.Dense(1))
#red_evaluation.add(keras.layers.Lambda(lambda x: x * 5))
red_evaluation.summary()
red_evaluation.compile(optimizer='SGD', loss='mean_squared_error', metrics=['accuracy', 'mae'])
red_evaluation.fit(atributos_entrenamiento, objetivo_entrenamiento,
                batch_size=256, epochs=10)
red_evaluation.evaluate(atributos_prueba, objetivo_prueba)

[[0.08333333 0.         0.         ... 0.5        0.5        0.5       ]
 [0.08333333 0.         0.25       ... 0.5        0.5        0.5       ]
 [0.08333333 0.         0.5        ... 1.         1.         1.        ]
 ...
 [1.         0.         0.         ... 1.         1.         1.        ]
 [1.         0.         0.25       ... 0.         0.         0.        ]
 [1.         0.         0.25       ... 0.         0.         0.        ]]
[[0.5]
 [0.5]
 [1. ]
 ...
 [1. ]
 [0. ]
 [0. ]]
[[0.5]
 [0.5]
 [1. ]
 ...
 [1. ]
 [0. ]
 [0. ]]
Model: "sequential_35"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_35 (Normaliza  (None, 31)               63        
 tion)                                                           
                                                                 
 dense_137 (Dense)           (None, 75)                2400      
                                         

[0.01639534905552864, 0.3298968970775604, 0.08413909375667572]

In [6]:
#Metodo LIME
import random
from sklearn.linear_model import Ridge



"""
N es el n´umero de permutaciones a realizar
f es el modelo a explicar
X′ ← {} muestras perturbadas
R ← {} representaciones
W ← {} las distancias entre la muestra x y sus perturbaciones
for 1
to
N do
Selecciona k atributos aleatoriamente
x′ ← una perturbaci´on de x donde se perturban los k atributos anteriores.
w ← la distancia entre x y x′
r ← la representaci´on de x′ respecto a x
X′ ← X′ ∪ x′
R ← R ∪ r
W ← R ∪ w
end for
Y ′ ← f(X′) las predicciones de las perturbaciones
G ← modelo ridge entrenado con R para predecir Y ′ y ponderando cada
muestra con W
return los par´ametros de G
"""

def explain_model(f, x, N,M):
    #f es el modelo a explicar
    #N es el número de permutaciones a realizar
    Xi = [] #muestras perturbadas
    R = [] #representaciones
    W = [] #Las distancias entre la muestra x y sus pertubaciones
    for i in range(N):
        k = random.randint(1, len(x)) #Escojo un número aletario que representa el número de los atributos a seleccionar
        perturbed_x = x.copy() #copio la muestra original
        for i in range(k): 
            perturbed_attr = random.choice(list(x.keys())) #Voy escojiento los atributos que perturbare de forma aleatoria
            mx = abs(max([a[perturbed_attr] for a in M])) #Para el atributo aletorio seleccionado calculo el valor máximo
            mn = abs(min([a[perturbed_attr] for a in M])) #Para el atributo aletorio seleccionado calculo el valor mínimo
            perturbed_x[perturbed_attr] = random.uniform(mn, mx) # pertubo el atributo con un valor aleatorio ente mn y mx
        w = abs(sum([x[attr] - perturbed_x[attr] for attr in x.keys()])) #Calculo la distancia entre el original y el perturbado
        r = {attr: 0  if x_perturbed[attr] = x[attr] else 1 for attr in x.keys()} #miramos la representacion del pertubado respecto al original utilizando un operador ternario como solemos hacer en java
        Xi.append(x_perturbed) #Acumulo en la lista los perturbados
        R.append(r)#Acumulamos en la lista de representaciones
        W.append(w)#Añadimos las distancias a la lista
    Y_perturbed = f(Xi)#aplicamos el modelo f
    G = Ridge().fit(R, Y_perturbed, sample_weight=W) #Utilizamos Ridge 
    return G.get_params() #Devolvemos los parametros


In [27]:
#METODO IDENTIDAD
import pandas
import numpy as np
from scipy.spatial.distance import cdist
from scipy.stats import pearsonr
from scipy.special import lambertw
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# Creamos dos instancias de datos y sus respectivas explicaciones LIME
xa = np.array([0.5, 0.7, 0.1, 0.3])
xb = np.array([0.3, 0.9, 0.2, 0.1])
ea = np.array([0.2, 0.3, 0.1, 0.4])
eb = np.array([0.3, 0.4, 0.5, 0.1])

# Calculamos la distancia entre las instancias de datos
data_distance = cdist(xa.reshape(1,-1), xb.reshape(1,-1))

# Calculamos la distancia entre las explicaciones LIME
exp_distance = cdist(ea.reshape(1,-1), eb.reshape(1,-1))

# Verificamos si la métrica Identidad se cumple
if data_distance == 0 and exp_distance != 0:
    print("La métrica Identidad no se cumple para este par de instancias de datos y sus explicaciones LIME correspondientes.")
else:
    print("La métrica Identidad se cumple para este par de instancias de datos y sus explicaciones LIME correspondientes.")


La métrica Identidad se cumple para este par de instancias de datos y sus explicaciones LIME correspondientes.


In [ ]:
#METODO SEPARABILIDAD
# Creamos dos instancias de datos y sus respectivas explicaciones LIME
xa = np.array([0.5, 0.7, 0.1, 0.3])
xb = np.array([0.3, 0.9, 0.2, 0.1])
ea = np.array([0.2, 0.3, 0.1, 0.4])
eb = np.array([0.3, 0.4, 0.5, 0.1])

# Calculamos la distancia entre las instancias de datos
data_distance = cdist(xa.reshape(1,-1), xb.reshape(1,-1))

# Calculamos la distancia entre las explicaciones LIME
exp_distance = cdist(ea.reshape(1,-1), eb.reshape(1,-1))

# Verificamos si la métrica Separabilidad se cumple
if data_distance != 0 and exp_distance <= 0:
    print("La métrica Separabilidad no se cumple para este par de instancias de datos y sus explicaciones LIME correspondientes.")
else:
    print("La métrica Separabilidad se cumple para este par de instancias de datos y sus explicaciones LIME correspondientes.")


In [ ]:
#METODO ESTABILIDAD
# Creamos las instancias de datos y sus respectivas explicaciones LIME
x = np.array([[0.5, 0.7, 0.1, 0.3], [0.3, 0.9, 0.2, 0.1], [0.1, 0.2, 0.8, 0.6]])
ex = np.array([[0.2, 0.3, 0.1, 0.4], [0.3, 0.4, 0.5, 0.1], [0.1, 0.3, 0.6, 0.2]])

# Creamos las distancias entre las instancias de datos y sus respectivas explicaciones LIME
data_distances = cdist(x, x[0].reshape(1,-1))
exp_distances = cdist(ex, ex[0].reshape(1,-1))

# Calculamos el coeficiente de correlación de Pearson entre las distancias
correlation, _ = pearsonr(data_distances.flatten(), exp_distances.flatten())

# Verificamos si la métrica Estabilidad se cumple
if correlation > 0:
    print("La métrica Estabilidad se cumple para este conjunto de instancias de datos y sus explicaciones LIME correspondientes.")
else:
    print("La métrica Estabilidad no se cumple para este conjunto de instancias de datos y sus explicaciones LIME correspondientes.")


In [ ]:
#METODO COHERENCIA
# Creamos las instancias de datos y sus respectivas explicaciones LIME
x = np.array([[0.5, 0.7, 0.1, 0.3], [0.3, 0.9, 0.2, 0.1], [0.1, 0.2, 0.8, 0.6]])
ex = np.array([[0.2, 0.3, 0.1, 0.4], [0.3, 0.4, 0.5, 0.1], [0.1, 0.3, 0.6, 0.2]])

# Calculamos los pesos de las instancias de datos y sus respectivas explicaciones LIME
data_weights = np.exp(-np.linalg.norm(x - x.mean(axis=0), axis=1))
exp_weights = np.exp(-np.linalg.norm(ex - ex.mean(axis=0), axis=1))

# Calculamos los logaritmos naturales de los pesos y los sumamos
log_sum_data = np.sum(np.log(data_weights))
log_sum_exp = np.sum(np.log(exp_weights))

# Calculamos la métrica Coherencia
alpha = np.exp(log_sum_data - log_sum_exp + lambertw(0.0, -np.exp(log_sum_data - log_sum_exp)).real)
alpha /= len(x)

# Imprimimos el valor de la métrica Coherencia
print("El valor de la métrica Coherencia es:", alpha)


In [ ]:
#METODO COMPLETITUD
# Creamos las instancias de datos y sus respectivas explicaciones LIME
x = np.array([[0.5, 0.7, 0.1, 0.3], [0.3, 0.9, 0.2, 0.1], [0.1, 0.2, 0.8, 0.6]])
ex = np.array([[0.2, 0.3, 0.1, 0.4], [0.3, 0.4, 0.5, 0.1], [0.1, 0.3, 0.6, 0.2]])

# Calculamos los pesos de las instancias de datos y sus respectivas explicaciones LIME
data_weights = np.exp(-np.linalg.norm(x - x.mean(axis=0), axis=1))
exp_weights = np.exp(-np.linalg.norm(ex - ex.mean(axis=0), axis=1))

# Calculamos los valores de ei y pi
ei = np.sum(exp_weights)
pi = np.sum(data_weights)

# Calculamos la métrica Completitud
gamma = ei / pi

# Imprimimos el valor de la métrica Completitud
print("El valor de la métrica Completitud es:", gamma)


In [ ]:
#METODO CONGRUENCIA
# Creamos una lista de valores de coherencia para un conjunto de N muestras
alpha_values = [0.8, 0.6, 0.7, 0.9, 0.5]

# Calculamos el valor promedio de la coherencia
alpha = np.mean(alpha_values)

# Creamos una lista de valores de alpha_i para cada instancia de datos
alpha_i_values = [0.9, 0.7, 0.6, 0.8, 0.4]

# Calculamos el número de instancias de datos
N = len(alpha_i_values)

# Calculamos la suma de las diferencias cuadráticas entre alpha_i y alpha
diff_squares_sum = np.sum((np.array(alpha_i_values) - alpha) ** 2)

# Calculamos la métrica Congruencia
delta = np.sqrt((diff_squares_sum / N) / alpha)

# Imprimimos el valor de la métrica Congruencia
print("El valor de la métrica Congruencia es:", delta)


In [ ]:
#METODO SELECTIVIDAD
# Cargamos el dataset de cáncer de mama de scikit-learn
data = load_breast_cancer()

# Definimos las características y la variable objetivo
X = data['data']
y = data['target']

# Creamos un modelo de regresión logística
model = LogisticRegression()

# Entrenamos el modelo
model.fit(X, y)

# Obtenemos las predicciones del modelo en el conjunto de datos completo
y_pred = model.predict_proba(X)[:, 1]

# Calculamos el AUC para el modelo entrenado en el conjunto de datos completo
auc_full = roc_auc_score(y, y_pred)

# Ordenamos las características de más a menos relevantes según el modelo
idx_sorted = np.argsort(np.abs(model.coef_))[0][::-1]

# Creamos una lista para almacenar los AUC para cada característica eliminada
auc_scores = []

# Iteramos sobre las características de más a menos relevantes según el modelo
for i in range(X.shape[1]):
    # Establecemos la característica i en cero
    X_test = X.copy()
    X_test[:, idx_sorted[i]] = 0
    
    # Obtenemos las predicciones del modelo en el conjunto de datos con la característica i eliminada
    y_pred_test = model.predict_proba(X_test)[:, 1]
    
    # Calculamos el AUC para el modelo entrenado en el conjunto de datos con la característica i eliminada
    auc_test = roc_auc_score(y, y_pred_test)
    
    # Almacenamos el AUC para la característica i eliminada en la lista de AUC
    auc_scores.append(auc_test)

# Calculamos la métrica Selectividad como el área bajo la curva ROC para el gráfico de AUC vs características eliminadas
selectivity = np.trapz(auc_scores, dx=1)

# Imprimimos el valor de la métrica Selectividad
print("El valor de la métrica Selectividad es:", selectivity)
